In [2]:
import cifar_gpu_fun
reload(cifar_gpu_fun)
from cifar_gpu_fun import *
from numba import jit
import bcd
import sys
%pylab inline

Using gpu device 0: Tesla K20c (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5103)
/data/vaishaal/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


#### Define Some Constants 

In [7]:
NUM_FEATURE_BATCHES=16
DATA_BATCH_SIZE=(1280)
FEATURE_BATCH_SIZE=(1024)
NUM_TRAIN = 50000
NUM_TEST = 10000
NUM_CLASSES = 10
NUM_FEATURES = FEATURE_BATCH_SIZE * NUM_FEATURE_BATCHES * 8
POOL_TYPE ='avg'
FILTER_GEN ='empirical'
BANDWIDTH = 1.0
LAMBDAS = [0.01, 0.1, 1.0, 0.001]
#LAMBDAS = [0.01]
CUDA_CONVNET = True
SCALE = 55.0
BIAS = 1.25
MIN_VAR_TOL = 1e-4
TOT_FEAT = FEATURE_BATCH_SIZE*NUM_FEATURE_BATCHES
NAN = float('nan')
saved = 0

In [4]:
np.random.seed(10)                                                                                                                                                                   
(XTrain, labelsTrain), (XTest, labelsTest) = load_cifar_processed()
patches = patchify_all_imgs(XTrain, (6,6), pad=False)

(50000, 3, 32, 32)


In [5]:
@jit(nopython=True)
def rbf(K, gamma):
    for x in range(K.shape[0]):
        for y in range(K.shape[1]):
            K[x,y] = math.exp(gamma*K[x,y])
            
    return K

@jit(nopython=True)
def arccos(K, XTrainNorms, XTestNorms):
    for x in range(K.shape[0]):
        for y in range(K.shape[1]):
            testNorm = np.sqrt(XTestNorms[x])
            trainNorm = np.sqrt(XTrainNorms[y]) 
            if (trainNorm == 0):
                trainNorm = 1
            if (testNorm == 0):
                testNorm = 1 
            cos = min(K[x,y]/(testNorm*trainNorm), 1)
            theta = np.arccos(cos)
            J = math.sin(theta) - (math.pi - theta)*math.cos(theta)
            K[x,y] = 1.0/math.pi * J
    return K
    


def computeRBFGramMatrix(K, XTrainNorms, XTestNorms, gamma=1, gamma_sample=10000):
    
    XTrainNorms = XTrainNorms.reshape(XTrainNorms.shape[0], 1)
    XTestNorms = XTestNorms.reshape(XTestNorms.shape[0], 1)
    print "TURNING K -> DISTANCE"
    K *= -2
    K += XTrainNorms.T
    K += XTestNorms 
    if (gamma == None):
        print "Calculating gamma"
        samples = numpy.random.choice(K.shape[0], gamma_sample*2, replace=False)
        x1 = samples[:gamma_sample]
        x2 = samples[gamma_sample:]
        sample_d = K[x1, x2]
        print "Sample d shape ", sample_d.shape
        median = numpy.median(sample_d)
        gamma = 2.0/median
        print gamma
    gamma = -1.0 * gamma

    print "Computing RBF"
    return rbf(K, gamma), -1.0*gamma

def saveGramMatrix(KTrain, KTest, save_id=saved):
    global saved
    KFile = open("/data/vaishaal/cifar_kernels/cifar_kernel_{0}".format(save_id), "w+")
    np.savez(KFile, train=KTrain, test=KTest)
    KFile.close()
    saved += 1
    
def computeDistanceMatrix(XTest, XTrain):
    XTrain = XTrain.reshape(XTrain.shape[0], -1)
    XTest = XTest.reshape(XTest.shape[0], -1)
    XTrain_norms = (np.linalg.norm(XTrain, axis=1) ** 2)[:, np.newaxis]
    XTest_norms = (np.linalg.norm(XTest, axis=1) ** 2)[:, np.newaxis]
    K = XTest.dot(XTrain.T)
    K *= -2
    K += XTrain_norms.T
    K += XTest_norms  
    return K
    
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [34]:
VALID_3_POOLS = [(15,6)]

In [6]:
results = {}

In [1]:
sys.stdout = terminal_stdout
sys.stderr = terminal_stderr
#sys.stdout = notebook_stdout
#sys.stderr = notebook_stderr
accs = []
results = {}
for param in VALID_3_POOLS:
    pool_size, pool_stride = param
    filter_gen = make_empirical_filter_gen(patches, labelsTrain, MIN_VAR_TOL, seed=3)
    KTrain, KTest, normsTrain, normsTest, filters = convolveAndAccumulateGramAsync(XTrain, XTest, 
                                                                     labelsTrain, labelsTest, filter_gen,
                                                                     num_feature_batches=NUM_FEATURE_BATCHES, pool_size=pool_size, pool_stride=pool_stride, sync=40)
    
    result = {}
    for reg in LAMBDAS:
        acc = trainAndEvaluateDualModel(KTrain, KTest, labelsTrain, labelsTest, reg=reg)
        print "RESULT ", pool_size,pool_stride, reg, acc
        result[reg] = acc
    results[param] = result


NameError: name 'terminal_stdout' is not defined

In [10]:
print results

In [46]:

result = {}
for reg in LAMBDAS:
    acc = trainAndEvaluateDualModel(KTrain, KTest, labelsTrain, labelsTest, reg=reg)
    print "RESULT ", pool_size,pool_stride, reg, acc
    result[reg] = acc


KeyboardInterrupt: 

In [56]:
sys.stdout = notebook_stdout
sys.stderr = notebook_stderr
for param in results.keys():
    pool_size, pool_stride = param
    for reg in results[param].keys():
        train, test = results[param][reg]
        print pool_size,pool_stride,pool_size - pool_stride,train,test,"0",reg


15 6 9 0.84086 0.7699 0 1.0
15 6 9 1.0 0.812 0 0.001
15 6 9 0.963 0.8233 0 0.1
15 6 9 0.99974 0.8353 0 0.01


In [106]:
gammas = [0.005]
regs = [0]
accs = []

In [22]:
gamma = 0.001

In [107]:
sys.stdout = notebook_stdout
sys.stderr = notebook_stderr
for gamma in gammas:
    print "GAMMA"
    KTrainRBF = KTrain.copy()
    KTestRBF = KTest.copy()
    KTrainRBF, train_gamma = computeRBFGramMatrix(KTrainRBF, normsTrain, normsTrain, gamma=gamma)
    KTestRBF, _ = computeRBFGramMatrix(KTestRBF, normsTrain, normsTest, gamma=train_gamma)
    for reg in regs:
        try:
            print("Starting Train","reg", reg, "gamma", gamma)
            sys.stdout.flush()
            %time train,test = trainAndEvaluateDualModel(KTrainRBF, KTestRBF, labelsTrain, labelsTest, reg=reg)
            print train,test,1,reg,gamma
            sys.stdout.flush()
            accs.append((gamma,reg,train,test))
        except:
            print("Singular Matrix")
            accs.append(0)


GAMMA
TURNING K -> DISTANCE
Computing RBF
TURNING K -> DISTANCE
Computing RBF
('Starting Train', 'reg', 0, 'gamma', 0.005)
Learning Dual Model better
REG IS  0
CPU times: user 24min 58s, sys: 2min 3s, total: 27min 2s
Wall time: 2min 51s
1.0 0.8305 1 0 0.005


In [48]:
sys.stdout = notebook_stdout
sys.stderr = notebook_stderr
for i, acc in enumerate(accs):
    gamma,reg,train, test = acc
    print train,test,1,reg,gamma

1.0 0.8436 1 1e-08 0.0005
1.0 0.8456 1 1e-06 0.0005
1.0 0.852 1 5e-06 0.0005
0.99988 0.8522 1 1e-05 0.0005
0.99086 0.8431 1 5e-05 0.0005
0.97196 0.8342 1 0.0001 0.0005
0.88968 0.8012 1 0.0005 0.0005
0.84992 0.7808 1 0.001 0.0005
0.7665 0.7285 1 0.005 0.0005
0.73396 0.7085 1 0.01 0.0005
0.66548 0.6544 1 0.05 0.0005
0.63952 0.6279 1 0.1 0.0005
1.0 0.8447 1 1e-08 0.001
1.0 0.8471 1 1e-06 0.001
1.0 0.8506 1 5e-06 0.001
1.0 0.853 1 1e-05 0.001
0.99846 0.8502 1 5e-05 0.001
0.99192 0.8445 1 0.0001 0.001
0.932 0.82 1 0.0005 0.001
0.8935 0.8049 1 0.001 0.001
0.80504 0.7539 1 0.005 0.001
0.77008 0.7311 1 0.01 0.001
0.69596 0.6787 1 0.05 0.001
0.66698 0.6539 1 0.1 0.001
1.0 0.8471 1 1e-08 0.0025
1.0 0.8479 1 1e-06 0.0025
1.0 0.8492 1 5e-06 0.0025
1.0 0.8511 1 1e-05 0.0025
0.99994 0.8539 1 5e-05 0.0025
0.99926 0.851 1 0.0001 0.0025
0.9781 0.839 1 0.0005 0.0025
0.95076 0.8302 1 0.001 0.0025
0.86102 0.7884 1 0.005 0.0025
0.8237 0.7653 1 0.01 0.0025
0.74074 0.7138 1 0.05 0.0025
0.70864 0.6897 1 0.1 0

In [91]:
KTrain_128 = np.load("/data/vaishaal/KTrain_128.npy")
KTest_128 = np.load("/data/vaishaal/KTest_128.npy")
normsTrain_128 = np.load("/data/vaishaal/norms_train_128.npy")
normsTest_128 = np.load("/data/vaishaal/norms_test_128.npy")

In [92]:
KTrain_fuse = KTrain_128 + KTrain
KTest_fuse = KTest_128 + KTest
normsTrain_fuse = normsTrain_128 + normsTrain
normsTest_fuse = normsTest_128 + normsTest

In [102]:
acc1 = trainAndEvaluateDualModel(KTrain, KTest, labelsTrain, labelsTest, reg=0.01)

Learning Dual Model better
REG IS  0.01


In [103]:
acc1

(0.99973999999999996, 0.83530000000000004)

In [99]:
acc2 = trainAndEvaluateDualModel(KTrain_fuse, KTest_fuse, labelsTrain, labelsTest, reg=0.05)
print acc2

Learning Dual Model better
REG IS  0.05
(0.99883999999999995, 0.82879999999999998)


In [96]:
print acc2

(1.0, 0.83109999999999995)


In [40]:
np.save("/data/vaishaal/KTrain_1024_patches", KTrain)

In [39]:
np.save("/data/vaishaal/KTest_1024_patches", KTest)

In [38]:
np.save("/data/vaishaal/norms_train_1024_patches", normsTrain)

In [37]:
np.save("/data/vaishaal/norms_test_1024_patches", normsTest)

In [23]:
np.save("/data/vaishaal/labels_train", labelsTrain)

In [24]:
np.save("/data/vaishaal/labels_test", labelsTest)

In [103]:
del KTrain_128

In [104]:
del KTrain_fuse

In [25]:
KTrain.shape

(50000, 50000)

In [16]:
patches

(50000, 729, 6, 6, 3)

In [72]:
KTrainRBF = KTrain.copy()
KTestRBF = KTest.copy()
def kernel_to_distance(K, XTrainNorms, XTestNorms):
    XTrainNorms = XTrainNorms.reshape(XTrainNorms.shape[0], 1)
    XTestNorms = XTestNorms.reshape(XTestNorms.shape[0], 1)
    print "TURNING K -> DISTANCE"
    K *= -2
    K += XTrainNorms.T
    K += XTestNorms 
    return K

def estimate_gamma(D,gamma_sample=10000):
    print "Calculating gamma"
    samples = numpy.random.choice(D.shape[0], gamma_sample*2, replace=False)
    x1 = samples[:gamma_sample]
    x2 = samples[gamma_sample:]
    sample_d = D[x1, x2]
    print "Sample d shape ", sample_d.shape
    median = np.median(sample_d)
    gamma = 2.0/median
    return gamma
D = kernel_to_distance(KTrainRBF, normsTrain, normsTrain)

TURNING K -> DISTANCE


In [91]:
pts = np.random.choice(D.shape[0], 1000*2, replace=False)
x1 = pts[:1000]
x2 = pts[1000:]
sample_d = D[x1, x2]
print "Sample d shape ", sample_d.shape
median = np.median(sample_d)
gamma = 2.0/median


Sample d shape  (1000,)


In [101]:
computeRBFGramMatrix(KTrain, XTrainNorms, XTestNorms, gamma=1, gamma_sample=10000)

In [4]:
import sys
print sys.stdout

In [95]:
KTrainRBF

array([[ 33.20146561,  36.28236389,  15.55484772, ...,  17.90564346,
         25.23268318,  29.74692154],
       [ 36.28236389,  44.59234619,  17.80164909, ...,  20.59594727,
         30.45558167,  34.82453918],
       [ 15.55484772,  17.80164909,   9.16023064, ...,   9.84381771,
         11.05340576,  15.00150013],
       ..., 
       [ 17.90564346,  20.59594727,   9.84381771, ...,  11.95548248,
         13.03578663,  17.35746956],
       [ 25.23268318,  30.45558167,  11.05340576, ...,  13.03578663,
         25.01102257,  24.62996674],
       [ 29.74692154,  34.82453918,  15.00150013, ...,  17.35746956,
         24.62996674,  31.18713188]])

In [97]:
KTrain

array([[  0.00000000e+00,   5.22908783e+00,   1.12520008e+01, ...,
          9.34566212e+00,   7.74712181e+00,   4.89475441e+00],
       [  5.22908783e+00,   7.62939453e-06,   1.81492825e+01, ...,
          1.53559389e+01,   8.69220924e+00,   6.13040352e+00],
       [  1.12520008e+01,   1.81492825e+01,   0.00000000e+00, ...,
          1.42807865e+00,   1.20644417e+01,   1.03443623e+01],
       ..., 
       [  9.34566212e+00,   1.53559389e+01,   1.42807865e+00, ...,
          1.90734863e-06,   1.08949327e+01,   8.42767620e+00],
       [  7.74712181e+00,   8.69220924e+00,   1.20644417e+01, ...,
          1.08949327e+01,   0.00000000e+00,   6.93822098e+00],
       [  4.89475441e+00,   6.13040352e+00,   1.03443623e+01, ...,
          8.42767620e+00,   6.93822098e+00,   0.00000000e+00]])

In [104]:
KTrain

array([[ 33.20146561,  36.28236389,  15.55484772, ...,  17.90564346,
         25.23268318,  29.74692154],
       [ 36.28236389,  44.59234619,  17.80164909, ...,  20.59594727,
         30.45558167,  34.82453918],
       [ 15.55484772,  17.80164909,   9.16023064, ...,   9.84381771,
         11.05340576,  15.00150013],
       ..., 
       [ 17.90564346,  20.59594727,   9.84381771, ...,  11.95548248,
         13.03578663,  17.35746956],
       [ 25.23268318,  30.45558167,  11.05340576, ...,  13.03578663,
         25.01102257,  24.62996674],
       [ 29.74692154,  34.82453918,  15.00150013, ...,  17.35746956,
         24.62996674,  31.18713188]])

In [105]:
normsTrain

array([ 33.20146561,  44.59235001,   9.16023064, ...,  11.95548344,
        25.01102257,  31.18713188], dtype=float32)